In [1]:
import azureml.core 
print(azureml.core.VERSION)

1.21.0


In [10]:
!pip install --upgrade --upgrade-strategy eager azureml-sdk

Requirement already up-to-date: azureml-sdk in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.21.0)
Requirement already up-to-date: azureml-core~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0.post1)
Requirement already up-to-date: azureml-train~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: azureml-pipeline~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: azureml-train-automl-client~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: azureml-dataset-runtime[fuse]~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: backports.tempfile in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-core~=1.21.0->azureml

In [11]:
!pip install --upgrade azureml-sdk[automl]

Requirement already up-to-date: azureml-sdk[automl] in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.21.0)


In [1]:
 conda update -n base -c defaults conda

Solving environment: - \ | / - \ done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [14]:
from azureml.core.environment import Environment
Environment(name="vaenv")
myenv = Environment(name="vaenv")

In [15]:
from azureml.core.conda_dependencies import CondaDependencies


dependencies = CondaDependencies()
dependencies.add_pip_package("scikit-learn")
dependencies.add_pip_package("numpy==1.15.4")
myenv.python.conda_dependencies=dependencies

In [16]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name='Toria_workspace',
            subscription_id='784d8740-4c70-4d20-8f45-b2bfecf94029',
            resource_group='myresourcegroup',)
exp = Experiment(workspace=ws, name="Toria_workspace")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: Toria_workspace
Azure region: eastus2
Subscription id: 784d8740-4c70-4d20-8f45-b2bfecf94029
Resource group: myresourcegroup


In [18]:
ws = Workspace.from_config()
experiment_name = 'People_hyperdrive'

experiment=Experiment(ws, experiment_name)

In [19]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal,choice
import os
import shutil
from azureml.core import ScriptRunConfig
from azureml.core import Environment


# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.4)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "learning_rate": normal(10, 3),
    "keep_probability": uniform(0.05, 0.1),
    "--C": uniform(0.02, 1),
    "--max_iter": choice(500, 1000, 1500),

}) 

if "training" not in os.listdir():
    os.mkdir("training")
    print("Path created")
    shutil.copy("traintest.py","./training")
    print('copied')

#TODO: Create your estimator and hyperdrive config
compute_target = ws.compute_targets['vpacompute']
src = ScriptRunConfig(source_directory='.',
                      script='traintest.py',
                      arguments=["learning_rate", "keep_probability","--C","--max_iter"],
                      compute_target=compute_target,
                      environment = myenv)

hyperdrive_run_config = HyperDriveConfig(estimator=src,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=4)

Path created
copied


In [20]:
from azureml.core import Experiment

run = Experiment(ws,'People_hyperdrive').submit(src)
run.wait_for_completion(show_output=True)

RunId: People_hyperdrive_1612029678_454abe82
Web View: https://ml.azure.com/experiments/People_hyperdrive/runs/People_hyperdrive_1612029678_454abe82?wsid=/subscriptions/784d8740-4c70-4d20-8f45-b2bfecf94029/resourcegroups/myresourcegroup/workspaces/toria_workspace

Streaming azureml-logs/55_azureml-execution-tvmps_78280eed931b807a78d35e8b2e5bb17f433cec4a02d57c898b312ff8672f1b8e_d.txt

2021-01-30T18:01:39Z Starting output-watcher...
2021-01-30T18:01:39Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-01-30T18:01:39Z Executing 'Copy ACR Details file' on 10.0.0.5
2021-01-30T18:01:39Z Copy ACR Details file succeeded on 10.0.0.5. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_1977b0efb94134e883afa1fc00bc0dee
be8ec4e48d7f: Already exists
33b8b485aff0: Already exists
d887158cc58c: Already exists
05895bb28c18: Already exists
baf7ab26f516: Already exists
181182e3c9cf: Already exists
d584ef274e55: Already exists
c445dda5540

{'runId': 'People_hyperdrive_1612029678_454abe82',
 'target': 'vpacompute',
 'status': 'Completed',
 'startTimeUtc': '2021-01-30T18:01:40.933169Z',
 'endTimeUtc': '2021-01-30T18:03:00.066143Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'd79a5357-ae5d-4493-bbfe-447bdcc44dc2',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'traintest.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['learning_rate', 'keep_probability', '--C', '--max_iter'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'vpacompute',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'environment': {'name': 'vaenv',
   'version': 'Autosave_2

In [21]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [22]:
best_run = run.get_metrics()
print('\n Accuracy:', best_run['Accuracy'])
print('\n Regularization Strength:',best_run['Regularization Strength:'])
print('\n Max Iterations:',best_run['Max iterations:'])


 Accuracy: 0.9633333333333334

 Regularization Strength: 1.0

 Max Iterations: 1000


In [27]:
#TODO: Save the best model
import joblib
joblib.dump(best_run, 'outputs/best_hyperremoteml_model.pkl')

['outputs/best_hyperremoteml_model.pkl']

In [29]:
from azureml.core.model import Model
model = Model.register(model_path= 'outputs/best_hyperremoteml_model.pkl',
                       model_name="best_hyperremoteml_model",
                       tags={"data": "loans",  "method" : "sklearn"},
                       description="Predict loans with sklearn",
                       workspace=ws)
print(model)

Registering model best_hyperremoteml_model
Model(workspace=Workspace.create(name='toria_workspace', subscription_id='784d8740-4c70-4d20-8f45-b2bfecf94029', resource_group='myresourcegroup'), name=best_hyperremoteml_model, id=best_hyperremoteml_model:1, version=1, tags={'data': 'loans', 'method': 'sklearn'}, properties={})
